In [1]:
import pandas as pd
import tensorflow as tf
import re,string
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
import gensim
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics import confusion_matrix
import keras
from keras import layers,models
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Conv1D, Flatten , LSTM ,Dropout,Bidirectional
from tensorflow.keras import regularizers
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
tweets=pd.read_csv('Tweets.csv')

In [3]:
tweets_new= tweets[['airline_sentiment','text']]
tweets_new

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,positive,@AmericanAir thank you we got on a different f...
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...
14637,neutral,@AmericanAir Please bring American Airlines to...
14638,negative,"@AmericanAir you have my money, you change my ..."


In [4]:
def depure_data(data):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)

    # Remove new line characters
    data = re.sub('\s+', ' ', data)

    # Remove distracting single quotes
    data = re.sub("\'", "", data)
        
    return data

In [5]:
temp = []
#Splitting pd.Series to list
data_to_list = tweets_new['text'].values
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))
list(temp[:5])

['What said.',
 'plus youve added commercials to the experience... tacky.',
 'I didnt today... Must mean I need to take another trip!',
 'its really aggressive to blast obnoxious "entertainment" in your guests faces &amp; they have little recourse',
 'and its a really big bad thing about it']

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        

data_words = list(sent_to_words(temp))

print(data_words[:10])

[['what', 'said'], ['plus', 'youve', 'added', 'commercials', 'to', 'the', 'experience', 'tacky'], ['didnt', 'today', 'must', 'mean', 'need', 'to', 'take', 'another', 'trip'], ['its', 'really', 'aggressive', 'to', 'blast', 'obnoxious', 'entertainment', 'in', 'your', 'guests', 'faces', 'amp', 'they', 'have', 'little', 'recourse'], ['and', 'its', 'really', 'big', 'bad', 'thing', 'about', 'it'], ['seriously', 'would', 'pay', 'flight', 'for', 'seats', 'that', 'didnt', 'have', 'this', 'playing', 'its', 'really', 'the', 'only', 'bad', 'thing', 'about', 'flying', 'va'], ['yes', 'nearly', 'every', 'time', 'fly', 'vx', 'this', 'ear', 'worm', 'won', 'go', 'away'], ['really', 'missed', 'prime', 'opportunity', 'for', 'men', 'without', 'hats', 'parody', 'there'], ['well', 'didnt', 'but', 'now', 'do'], ['it', 'was', 'amazing', 'and', 'arrived', 'an', 'hour', 'early', 'youre', 'too', 'good', 'to', 'me']]


In [7]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))
print(data[:5])

['what said', 'plus youve added commercials to the experience tacky', 'didnt today must mean need to take another trip', 'its really aggressive to blast obnoxious entertainment in your guests faces amp they have little recourse', 'and its really big bad thing about it']


In [8]:
tweets_new.airline_sentiment.loc[tweets_new['airline_sentiment'] == 'neutral']=0
tweets_new.airline_sentiment.loc[tweets_new['airline_sentiment'] == 'negative']=1
tweets_new.airline_sentiment.loc[tweets_new['airline_sentiment'] == 'positive']=2   

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [36]:
tweets_new

,airline_sentiment,text
0,0,@VirginAmerica What @dhepburn said.
1,2,@VirginAmerica plus you've added commercials t...
2,0,@VirginAmerica I didn't today... Must mean I n...
3,1,@VirginAmerica it's really aggressive to blast...
4,1,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,2,@AmericanAir thank you we got on a different f...
14636,1,@AmericanAir leaving over 20 minutes Late Flig...
14637,0,@AmericanAir Please bring American Airlines to...
14638,1,"@AmericanAir you have my money, you change my ..."


In [9]:
y = tweets_new['airline_sentiment'].values     
labels = tf.keras.utils.to_categorical(y, 3, dtype="float32")

In [10]:
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...    0   51  209]
 [   0    0    0 ...    1    2  188]
 [   0    0    0 ...  140  135  177]
 ...
 [   0    0    0 ...  432  236    1]
 [   0    0    0 ...  116    8 2494]
 [   0    0    0 ...    2  166    5]]


In [11]:
X_train, X_test, y_train, y_test = train_test_split(tweets,labels, random_state=0)

In [61]:
embedding_dim=150

model1 = Sequential([         
    Embedding(max_words,embedding_dim, input_length= max_len),
    Conv1D(filters=64,kernel_size=8, activation='relu'),
    MaxPooling1D(),
    Conv1D(filters=32,kernel_size=4, activation='relu'),#, bias_regularizer=l2(0.01)),
    MaxPooling1D(),
    Flatten(),
    Dense(10,activation='relu',kernel_regularizer='l1'),
    Dense(3,activation='softmax')

])

In [97]:
model1.compile(loss= tf.keras.losses.CategoricalCrossentropy(),optimizer='adam',metrics=['accuracy'])

In [63]:
history1 = model1.fit(X_train,y_train,epochs=8,verbose=2)

Epoch 1/8
344/344 - 10s - loss: 1.1708 - accuracy: 0.6597
Epoch 2/8
344/344 - 10s - loss: 0.6747 - accuracy: 0.7072
Epoch 3/8
344/344 - 10s - loss: 0.6019 - accuracy: 0.7270
Epoch 4/8
344/344 - 10s - loss: 0.5264 - accuracy: 0.7781
Epoch 5/8
344/344 - 10s - loss: 0.4435 - accuracy: 0.8374
Epoch 6/8
344/344 - 11s - loss: 0.3764 - accuracy: 0.8716
Epoch 7/8
344/344 - 11s - loss: 0.3110 - accuracy: 0.8951
Epoch 8/8
344/344 - 11s - loss: 0.2664 - accuracy: 0.9116


In [64]:
loss,acc = model1.evaluate(X_test,y_test,verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 71.420765


In [14]:
embedding_dim=300

model = Sequential([         
    Embedding(max_words,embedding_dim, input_length= max_len),
    Conv1D(filters=64,kernel_size=8, activation='relu'),
    MaxPooling1D(),
    Conv1D(filters=32,kernel_size=4, activation='relu'),#, bias_regularizer=l2(0.01)),
    MaxPooling1D(),
    Flatten(),
    Dense(10,activation='relu',kernel_regularizer='l1'),
    Dense(3,activation='softmax')

])
model.compile(loss= tf.keras.losses.CategoricalCrossentropy(),optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train,y_train,epochs=8,verbose=2)

Epoch 1/8
344/344 - 39s - loss: 1.1531 - accuracy: 0.6600 - 39s/epoch - 113ms/step
Epoch 2/8
344/344 - 36s - loss: 0.6506 - accuracy: 0.7299 - 36s/epoch - 104ms/step
Epoch 3/8
344/344 - 36s - loss: 0.5378 - accuracy: 0.7980 - 36s/epoch - 106ms/step
Epoch 4/8
344/344 - 36s - loss: 0.4330 - accuracy: 0.8454 - 36s/epoch - 106ms/step
Epoch 5/8
344/344 - 37s - loss: 0.3347 - accuracy: 0.8821 - 37s/epoch - 106ms/step
Epoch 6/8
344/344 - 36s - loss: 0.2704 - accuracy: 0.9087 - 36s/epoch - 105ms/step
Epoch 7/8
344/344 - 35s - loss: 0.2298 - accuracy: 0.9230 - 35s/epoch - 103ms/step
Epoch 8/8
344/344 - 37s - loss: 0.2056 - accuracy: 0.9301 - 37s/epoch - 106ms/step


In [15]:
loss,acc = model.evaluate(X_test,y_test,verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 72.486341


new version

In [2]:
tweets=pd.read_csv('Tweets.csv')
tweets_new= tweets[['airline_sentiment','text']]
tweets_new

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,positive,@AmericanAir thank you we got on a different f...
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...
14637,neutral,@AmericanAir Please bring American Airlines to...
14638,negative,"@AmericanAir you have my money, you change my ..."


In [3]:
tweets_new.airline_sentiment.loc[tweets_new['airline_sentiment'] == 'neutral']=0
tweets_new.airline_sentiment.loc[tweets_new['airline_sentiment'] == 'negative']=1
tweets_new.airline_sentiment.loc[tweets_new['airline_sentiment'] == 'positive']=2   
tweets_new

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,airline_sentiment,text
0,0,@VirginAmerica What @dhepburn said.
1,2,@VirginAmerica plus you've added commercials t...
2,0,@VirginAmerica I didn't today... Must mean I n...
3,1,@VirginAmerica it's really aggressive to blast...
4,1,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,2,@AmericanAir thank you we got on a different f...
14636,1,@AmericanAir leaving over 20 minutes Late Flig...
14637,0,@AmericanAir Please bring American Airlines to...
14638,1,"@AmericanAir you have my money, you change my ..."


In [4]:
y = tweets_new['airline_sentiment'].values     
labels = tf.keras.utils.to_categorical(y, 3, dtype="float32")

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format('glove.6B\glove.6B.300d.txt', binary = False,no_header=True) 

In [6]:
model.vector_size

300

In [7]:
economy_vec = model['economy']
economy_vec[:20]

array([ 0.32368 , -0.14921 , -0.47491 ,  0.11804 ,  0.34319 ,  0.2301  ,
       -0.20581 ,  0.43724 ,  0.077597, -2.5859  , -0.13203 ,  0.06049 ,
       -0.36986 ,  0.41528 ,  0.47718 , -0.19045 ,  0.10687 , -0.4391  ,
        0.41823 ,  0.072872], dtype=float32)

In [8]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.stem.wordnet import WordNetLemmatizer

In [6]:
from string import digits 
stop=set(stopwords.words('english'))
def tweets_cleaning(x, remove_emojis=True, remove_stop_words=True):
    """Apply function to a """
    x = x.lower().strip()
    # romove urls
    url = re.compile(r'https?://\S+|www\.\S+')
    x = url.sub(r'',x)
    # remove html tags
    html = re.compile(r'<.*?>')
    x = html.sub(r'',x)
    # remove punctuation
    operator = str.maketrans('','',string.punctuation) #????
    x = x.translate(operator)
    # remove digits
    remove_digits = str.maketrans('', '', digits) 
    x = ' '.join([i.translate(remove_digits) for i in x.split()])
    
    if remove_emojis:
        x = x.encode('ascii', 'ignore').decode('utf8').strip()
    if remove_stop_words:
        x = ' '.join([word for word in x.split(' ') if word not in stop])
    return x

In [7]:
tweets_new['clean_text'] = tweets_new['text'].apply(tweets_cleaning)

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_29548/2709407075.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_new['clean_text'] = tweets_new['text'].apply(tweets_cleaning)


In [8]:
tweets_new

,airline_sentiment,text,clean_text
0,0,@VirginAmerica What @dhepburn said.,virginamerica dhepburn said
1,2,@VirginAmerica plus you've added commercials t...,virginamerica plus youve added commercials exp...
2,0,@VirginAmerica I didn't today... Must mean I n...,virginamerica didnt today must mean need take ...
3,1,@VirginAmerica it's really aggressive to blast...,virginamerica really aggressive blast obnoxiou...
4,1,@VirginAmerica and it's a really big bad thing...,virginamerica really big bad thing
...,...,...,...
14635,2,@AmericanAir thank you we got on a different f...,americanair thank got different flight chicago
14636,1,@AmericanAir leaving over 20 minutes Late Flig...,americanair leaving minutes late flight warni...
14637,0,@AmericanAir Please bring American Airlines to...,americanair please bring american airlines bla...
14638,1,"@AmericanAir you have my money, you change my ...",americanair money change flight dont answer ph...


In [11]:
tweets_new[tweets_new['clean_text'].apply(len)<1]

,airline_sentiment,text,clean_text


In [9]:
X_train, X_test, y_train, y_test = train_test_split(tweets_new['clean_text'],labels, random_state=0)

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
lst_text2seq= tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(lst_text2seq, 
                    maxlen=150, padding="post", truncating="post")

lst_text2seq2= tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(lst_text2seq2, 
                    maxlen=150, padding="post", truncating="post")

In [11]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
i=0
for word in model.index_to_key:
    if word in word_index.keys():
        embedding_vector = model[word]
        embedding_matrix[i] = embedding_vector
        i +=1

In [15]:
print(embedding_matrix.shape)

(11293, 300)


In [16]:
def model_with_w2v():
    input_layer = layers.Input((150, ))

    # Add the word embedding Layer
    embedding_layer_1 = Embedding(len(tokenizer.word_index)+1, 300,weights=[embedding_matrix],trainable=False)(input_layer)
    embedding_layer_2 = layers.SpatialDropout1D(0.5)(embedding_layer_1)
    conv_1 = Conv1D(filters=32,kernel_size=8, activation='relu')(embedding_layer_2)
    max_pool_1 = MaxPooling1D()(conv_1)
    conv_2 = Conv1D(filters=16,kernel_size=4, activation='relu')(max_pool_1)
    max_pool_2 = MaxPooling1D()(conv_2)
    flat = Flatten()(max_pool_2)
    dense_1 = Dense(10,activation='relu',kernel_regularizer='l1')(flat)
    dense_2 = Dense(3,activation='softmax')(dense_1)
    
    # Compile the model
    model_ = models.Model(inputs=input_layer, outputs=dense_2)
    model_.compile(loss= tf.keras.losses.CategoricalCrossentropy(),optimizer='adam',metrics=['accuracy'])
    return model_

In [17]:
model_with_w2v=model_with_w2v()
history2 = model_with_w2v.fit(X_train,y_train,epochs=16,verbose=2)

Epoch 1/16
344/344 - 8s - loss: 1.3750 - accuracy: 0.6323 - 8s/epoch - 22ms/step
Epoch 2/16
344/344 - 7s - loss: 0.8309 - accuracy: 0.6487 - 7s/epoch - 21ms/step
Epoch 3/16
344/344 - 7s - loss: 0.7966 - accuracy: 0.6576 - 7s/epoch - 21ms/step
Epoch 4/16
344/344 - 7s - loss: 0.7718 - accuracy: 0.6657 - 7s/epoch - 21ms/step
Epoch 5/16
344/344 - 7s - loss: 0.7478 - accuracy: 0.6700 - 7s/epoch - 21ms/step
Epoch 6/16
344/344 - 7s - loss: 0.7160 - accuracy: 0.6884 - 7s/epoch - 21ms/step
Epoch 7/16
344/344 - 7s - loss: 0.6901 - accuracy: 0.7075 - 7s/epoch - 21ms/step
Epoch 8/16
344/344 - 7s - loss: 0.6639 - accuracy: 0.7266 - 7s/epoch - 21ms/step
Epoch 9/16
344/344 - 7s - loss: 0.6406 - accuracy: 0.7349 - 7s/epoch - 21ms/step
Epoch 10/16
344/344 - 7s - loss: 0.6091 - accuracy: 0.7535 - 7s/epoch - 21ms/step
Epoch 11/16
344/344 - 7s - loss: 0.5879 - accuracy: 0.7639 - 7s/epoch - 21ms/step
Epoch 12/16
344/344 - 7s - loss: 0.5614 - accuracy: 0.7748 - 7s/epoch - 22ms/step
Epoch 13/16
344/344 - 7s 

In [18]:
loss,acc = model_with_w2v.evaluate(X_test,y_test,verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 69.726777


lstm version

In [2]:
tweets=pd.read_csv('Tweets.csv')
tweets_new= tweets[['airline_sentiment','text']]
def depure_data(data):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)

    # Remove new line characters
    data = re.sub('\s+', ' ', data)

    # Remove distracting single quotes
    data = re.sub("\'", "", data)
        
    return data

In [3]:
temp = []
#Splitting pd.Series to list
data_to_list = tweets_new['text'].values
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))
list(temp[:5])

['What said.',
 'plus youve added commercials to the experience... tacky.',
 'I didnt today... Must mean I need to take another trip!',
 'its really aggressive to blast obnoxious "entertainment" in your guests faces &amp; they have little recourse',
 'and its a really big bad thing about it']

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        

data_words = list(sent_to_words(temp))

print(data_words[:10])

[['what', 'said'], ['plus', 'youve', 'added', 'commercials', 'to', 'the', 'experience', 'tacky'], ['didnt', 'today', 'must', 'mean', 'need', 'to', 'take', 'another', 'trip'], ['its', 'really', 'aggressive', 'to', 'blast', 'obnoxious', 'entertainment', 'in', 'your', 'guests', 'faces', 'amp', 'they', 'have', 'little', 'recourse'], ['and', 'its', 'really', 'big', 'bad', 'thing', 'about', 'it'], ['seriously', 'would', 'pay', 'flight', 'for', 'seats', 'that', 'didnt', 'have', 'this', 'playing', 'its', 'really', 'the', 'only', 'bad', 'thing', 'about', 'flying', 'va'], ['yes', 'nearly', 'every', 'time', 'fly', 'vx', 'this', 'ear', 'worm', 'won', 'go', 'away'], ['really', 'missed', 'prime', 'opportunity', 'for', 'men', 'without', 'hats', 'parody', 'there'], ['well', 'didnt', 'but', 'now', 'do'], ['it', 'was', 'amazing', 'and', 'arrived', 'an', 'hour', 'early', 'youre', 'too', 'good', 'to', 'me']]


In [5]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))
print(data[:5])

['what said', 'plus youve added commercials to the experience tacky', 'didnt today must mean need to take another trip', 'its really aggressive to blast obnoxious entertainment in your guests faces amp they have little recourse', 'and its really big bad thing about it']


In [6]:
tweets_new.airline_sentiment.loc[tweets_new['airline_sentiment'] == 'neutral']=0
tweets_new.airline_sentiment.loc[tweets_new['airline_sentiment'] == 'negative']=1
tweets_new.airline_sentiment.loc[tweets_new['airline_sentiment'] == 'positive']=2   

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [7]:
tweets_new

,airline_sentiment,text
0,0,@VirginAmerica What @dhepburn said.
1,2,@VirginAmerica plus you've added commercials t...
2,0,@VirginAmerica I didn't today... Must mean I n...
3,1,@VirginAmerica it's really aggressive to blast...
4,1,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,2,@AmericanAir thank you we got on a different f...
14636,1,@AmericanAir leaving over 20 minutes Late Flig...
14637,0,@AmericanAir Please bring American Airlines to...
14638,1,"@AmericanAir you have my money, you change my ..."


In [8]:
y = tweets_new['airline_sentiment'].values     
labels = tf.keras.utils.to_categorical(y, 3, dtype="float32")

In [9]:
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...    0   51  209]
 [   0    0    0 ...    1    2  188]
 [   0    0    0 ...  140  135  177]
 ...
 [   0    0    0 ...  432  236    1]
 [   0    0    0 ...  116    8 2494]
 [   0    0    0 ...    2  166    5]]


In [10]:
X_train, X_test, y_train, y_test = train_test_split(tweets,labels, random_state=0)

In [33]:
embedding_dim=300
model_lstm_1 = Sequential([         
    Embedding(max_words,embedding_dim, input_length= max_len),
    LSTM(35,dropout=0.5, recurrent_dropout=0.2,return_sequences=False),
    Dense(10,activation='relu',kernel_regularizer='l1'),
    Dropout(0.5),
    Dense(3,activation='softmax')

])
model_lstm_1.compile(loss= tf.keras.losses.CategoricalCrossentropy(),optimizer='adam',metrics=['accuracy'])
history_lstm_1 = model_lstm_1.fit(X_train,y_train,epochs=4,verbose=2)

Epoch 1/4
344/344 - 160s - loss: 1.1009 - accuracy: 0.6608 - 160s/epoch - 466ms/step
Epoch 2/4
344/344 - 414s - loss: 0.7029 - accuracy: 0.7447 - 414s/epoch - 1s/step
Epoch 3/4
344/344 - 510s - loss: 0.6067 - accuracy: 0.7825 - 510s/epoch - 1s/step
Epoch 4/4
344/344 - 501s - loss: 0.5411 - accuracy: 0.8049 - 501s/epoch - 1s/step


In [34]:
loss,acc = model_lstm_1.evaluate(X_test,y_test,verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 78.961748


In [24]:
embedding_dim=300
model_multi_lstm = Sequential([         
    Embedding(max_words,embedding_dim, input_length= max_len),
    LSTM(10, return_sequences=True),
    LSTM(10,return_sequences=True),
    LSTM(10,dropout=0.2, recurrent_dropout=0.2,return_sequences=False),
    Dense(10,activation='relu',kernel_regularizer='l1'),
    Dropout(0.4),
    Dense(3,activation='softmax')

])
model_multi_lstm.compile(loss= tf.keras.losses.CategoricalCrossentropy(),optimizer='adam',metrics=['accuracy'])
history_lstm_multi = model_multi_lstm.fit(X_train,y_train,epochs=10,verbose=2)

Epoch 1/10
344/344 - 78s - loss: 0.9811 - accuracy: 0.6496 - 78s/epoch - 228ms/step
Epoch 2/10
344/344 - 74s - loss: 0.7173 - accuracy: 0.7189 - 74s/epoch - 215ms/step
Epoch 3/10
344/344 - 74s - loss: 0.6142 - accuracy: 0.7563 - 74s/epoch - 215ms/step
Epoch 4/10
344/344 - 75s - loss: 0.5432 - accuracy: 0.7968 - 75s/epoch - 217ms/step
Epoch 5/10
344/344 - 74s - loss: 0.4840 - accuracy: 0.8339 - 74s/epoch - 216ms/step
Epoch 6/10
344/344 - 74s - loss: 0.4203 - accuracy: 0.8669 - 74s/epoch - 216ms/step
Epoch 7/10
344/344 - 74s - loss: 0.3809 - accuracy: 0.8904 - 74s/epoch - 216ms/step
Epoch 8/10
344/344 - 74s - loss: 0.3477 - accuracy: 0.9067 - 74s/epoch - 216ms/step
Epoch 9/10
344/344 - 74s - loss: 0.3165 - accuracy: 0.9168 - 74s/epoch - 216ms/step
Epoch 10/10
344/344 - 74s - loss: 0.2959 - accuracy: 0.9258 - 74s/epoch - 216ms/step


In [25]:
loss,acc = model_multi_lstm.evaluate(X_test,y_test,verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 78.989071


In [ ]:
def multi_lstm(optimizer='adam',
               dropout1=0.5,
               dropout2=0.2,
               recurrent_dropout=0.2,
               unit1=10,
               unit2=10,
               ):
    model = Sequential()
    model.add(Embedding(max_words,300, input_length= max_len))
    model.add(LSTM(unit1,dropout=dropout1, recurrent_dropout=recurrent_dropout, return_sequences=True))
    model.add(LSTM(unit2,return_sequences=True))
    model.add(LSTM(unit2,dropout=dropout2, recurrent_dropout=recurrent_dropout,return_sequences=False))
    model.add(Dense(10,activation='relu',kernel_regularizer='l1'))
    model.add(Dropout(dropout1))
    model.add(Dense(3,activation='softmax'))

    model.compile(loss= tf.keras.losses.CategoricalCrossentropy(),optimizer=optimizer,metrics=['accuracy'])

    return model

lstm_model = KerasClassifier(build_fn=multi_lstm,verbose=0)

param_grid = {
    'optimizer':['adam','adagrad'],
    'epochs':[8,10,12,15],
    'dropout1':[0.2,0.5,0.7],
    'dropout2':[0.2,0.5,0.7],
    'recurrent_dropout':[0.2,0.4],
    'unit1':[8,10,15,20],
    'unit2':[8,10,15,20]
}

grid = GridSearchCV(lstm_model, cv=3, param_grid=param_grid,n_jobs =-1)
result=grid.fit(X_train, y_train)

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_20644/4136055292.py:21: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  lstm_model = KerasClassifier(build_fn=multi_lstm,verbose=0)


In [ ]:
print("Best: %f using %s" % (result.best_score_, result.best_params_))
params = result.cv_results_['params']